In [1]:
import tensorflow as tf

# MNIST 데이터를 다운로드 합니다.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("dataOneHot/", one_hot=True)

W0806 15:39:37.344661 140342562817792 deprecation.py:323] From <ipython-input-1-e5ed91b8b8cc>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0806 15:39:37.345591 140342562817792 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0806 15:39:37.346039 140342562817792 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting dataOneHot/train-images-idx3-ubyte.gz
Extracting dataOneHot/train-labels-idx1-ubyte.gz
Extracting dataOneHot/t10k-images-idx3-ubyte.gz


W0806 15:39:37.576839 140342562817792 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting dataOneHot/t10k-labels-idx1-ubyte.gz


In [2]:
# 입력값과 출력값을 받기 위한 플레이스홀데를 정의합니다.
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

In [3]:
# 변수들을 설정하고 소프트맥스 회귀 모델을 정의합니다.
W = tf.Variable(tf.zeros(shape=[784,10]), name='W')
b = tf.Variable(tf.zeros(shape=[10]), name='b')
logits = tf.matmul(x, W) + b
y_pred = tf.nn.softmax(logits)

In [4]:
# cross-engropy 손실 함수와 옵티마이저를 정의합니다.
#loss_function = -tf.reduce_sum(y*tf.log(y_pred), reduction_indices=[1])
loss_function = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y)
loss = tf.reduce_mean(loss_function)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# 텐서보드를 위한 요약 정보(scalar)를 정의합니다.
tf.summary.scalar("Loss", loss)

<tf.Tensor 'Loss:0' shape=() dtype=string>

In [5]:
# 세션을 열고 변수들에 초기값을 할당합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 텐서보드 요약정보들을 하나로 합칩니다.
merged = tf.summary.merge_all()
# 텐서보드 summary 정보들을 저장할 폴더 경로를 설정합니다.
tensorboard_writer = tf.summary.FileWriter('./tensorboard_log', sess.graph)

In [6]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y:batch_ys})
    summary = sess.run(merged, feed_dict={x: batch_xs, y: batch_ys})
    tensorboard_writer.add_summary(summary, i)
    if i%1000==999:
        print("#",i+1,": ", sess.run(loss_function, feed_dict={x:batch_xs, y:batch_ys}))

# 1000 :  [7.5994030e-02 1.9124330e-03 1.5160269e-03 1.5694959e-01 1.7504447e+00
 9.2326840e-03 2.7261549e-01 4.5598850e-02 8.2840100e-02 1.2579288e-01
 1.2820087e-02 1.1657989e-04 2.9350426e-02 9.5301531e-02 2.2208791e-02
 9.6875541e-03 2.5663935e-02 9.6235722e-01 9.1625601e-02 1.2782983e-03
 3.6028018e-03 4.3780194e-03 3.7637367e-03 6.3251071e-03 3.8372627e-01
 2.2293495e-03 1.0206257e-03 2.3125010e-02 7.9278508e-04 1.5030465e-02
 5.7115570e-02 1.5903290e-01 1.4999272e-01 2.5134892e+00 4.8664998e-02
 3.6998584e+00 4.1229855e-02 2.3916948e-03 2.3317540e-02 2.7020394e-03
 5.0043460e-04 2.0761434e-02 5.1153004e-02 8.0297990e-03 3.1351552e-05
 3.6534175e-02 2.4077378e-04 3.2016212e-01 1.4975670e-01 6.8241727e-01
 2.2879836e-01 1.5382792e-01 1.9470559e-03 2.4813421e-02 5.5764117e-03
 4.5610130e-02 4.0326471e+00 5.5305716e-02 6.9010006e-03 9.8700417e-05
 4.6447958e-03 3.3405647e-02 4.8494320e-02 7.8034379e-02 1.9026850e+00
 2.9819816e-01 7.2817689e-01 2.0404669e-02 1.0579572e-02 5.2010730e

In [7]:
# 학습이 끝나면 학습된 모델의 정확도를 출력합니다.
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('정확도: %f' %sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels}))

정확도: 0.922300


In [8]:
sess.close()